In [11]:
import pandas as pd
import os

DATA_DIR = '/fsx/jdelavande/benchlab/videos/data'
PREFIX = 'exp_'

dfs = []
for csv in os.listdir(DATA_DIR):
    if csv.endswith('.csv') and csv.startswith(PREFIX) and not 'codecarbon' in csv:
        df = pd.read_csv(os.path.join(DATA_DIR, csv))
        df['name_csv'] = csv
        df['model_name'] = csv.split('_')[1].split('_')[0]
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['energy_generate_gpu'] = combined_df.apply(
    lambda row: row['energy_generate'] if pd.notnull(row['energy_generate']) else row['energy_generate_gpu'],
    axis=1
)
combined_df.drop(columns=['energy_generate'], inplace=True)
# codecarbon_version,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
# Linux-5.15.0-1048-aws-x86_64-with-glibc2.31,3.10.16,3.0.2,8,AMD EPYC 7R13 Processor,1,1 x NVIDIA H100 80GB HBM3,-77.4903,39.0469,256,machine,N,1.0
combined_df['runs'] = 5
combined_df['cpu_count'] = 8
combined_df['cpu_model'] = 'AMD EPYC 7R13 Processor'
combined_df['gpu_count'] = 1
combined_df['gpu_model'] = 'NVIDIA A100 80GB HBM3'




In [12]:
combined_df['model_name'].unique()

array(['Mochi-1-preview', 'CogVideoX-5b', 'WAN2.1-T2V-1.3B-Diffusers',
       'AnimateDiff', 'CogVideoX-2b', 'LTX-Video-0.9.7-dev',
       'WAN2.1-T2V-14B-Diffusers'], dtype=object)

In [13]:
combined_df.columns

Index(['model_name', 'duration_generate', 'energy_generate_cpu',
       'energy_generate_ram', 'prompt', 'num_frames', 'runs', 'out_video',
       'out_csv', 'fps', 'warmup', 'output_path', 'name_csv', 'steps',
       'guidance_scale', 'negative_prompt', 'height', 'width', 'adapter_repo',
       'adapter_ckpt', 'base_model', 'out_gif', 'duration_upsample',
       'duration_denoise', 'energy_generate_gpu', 'energy_upsample_gpu',
       'energy_denoise_gpu', 'energy_upsample_cpu', 'energy_denoise_cpu',
       'energy_upsample_ram', 'energy_denoise_ram', 'upsample_model_name',
       'downscaled_height', 'downscaled_width', 'generate_steps',
       'denoise_steps', 'denoise_strength', 'decode_timestep',
       'image_cond_noise_scale', 'cpu_count', 'cpu_model', 'gpu_count',
       'gpu_model'],
      dtype='object')

In [14]:
combined_df[['name_csv', 'model_name', 'duration_generate', 'energy_generate_gpu', 'energy_generate_cpu', 'energy_generate_ram', 'prompt']].sort_values(
    by=['model_name', 'prompt'])

,name_csv,model_name,duration_generate,energy_generate_gpu,energy_generate_cpu,energy_generate_ram,prompt
35,exp_AnimateDiff_results_prompt5_2025-06-18_12-...,AnimateDiff,0.687965,0.000111,0.000015,0.000008,A cyberpunk hacker in a dark alleyway with hol...
51,exp_AnimateDiff_results_prompt3_2025-06-18_12-...,AnimateDiff,0.686887,0.000112,0.000020,0.000008,A dense jungle with exotic animals and mist
43,exp_AnimateDiff_results_prompt1_2025-06-18_12-...,AnimateDiff,0.687201,0.000112,0.000015,0.000008,"A futuristic cityscape at night, neon lights r..."
7,exp_AnimateDiff_results_prompt4_2025-06-18_12-...,AnimateDiff,0.697821,0.000110,0.000024,0.000008,A majestic dragon flying over snowy mountains
39,exp_AnimateDiff_results_prompt7_2025-06-18_12-...,AnimateDiff,0.693765,0.000113,0.000022,0.000008,A medieval castle on a cliff during a thunders...
...,...,...,...,...,...,...,...
60,exp_WAN2.1-T2V-14B-Diffusers_results_prompt10_...,WAN2.1-T2V-14B-Diffusers,489.694768,0.093814,0.006122,0.005170,A mystical forest with glowing mushrooms and f...
55,exp_WAN2.1-T2V-14B-Diffusers_results_prompt8_2...,WAN2.1-T2V-14B-Diffusers,489.884060,0.093803,0.010051,0.005172,A peaceful rural village in spring with bloomi...
22,exp_WAN2.1-T2V-14B-Diffusers_results_prompt2_2...,WAN2.1-T2V-14B-Diffusers,489.559194,0.093766,0.008953,0.005169,A serene beach at sunrise with gentle waves
45,exp_WAN2.1-T2V-14B-Diffusers_results_prompt9_2...,WAN2.1-T2V-14B-Diffusers,489.528463,0.093777,0.010649,0.005168,A spaceship approaching a giant alien planet


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sélection des colonnes utiles
combined_df_copy = combined_df.copy()
combined_df_copy['energy_generate_gpu'] = combined_df_copy.apply(
    lambda row: (row['energy_generate_gpu'] + row['energy_upsample_gpu'] + row['energy_denoise_gpu']) if row['model_name'] == 'LTX-Video-0.9.7-dev' else row['energy_generate_gpu'],
    axis=1
)
combined_df_copy['energy_generate_cpu'] = combined_df_copy.apply(
    lambda row: (row['energy_generate_cpu'] + row['energy_upsample_cpu'] + row['energy_denoise_cpu']) if row['model_name'] == 'LTX-Video-0.9.7-dev' else row['energy_generate_cpu'],
    axis=1
)
combined_df_copy['energy_generate_ram'] = combined_df_copy.apply(
    lambda row: (row['energy_generate_ram'] + row['energy_upsample_ram'] + row['energy_denoise_ram']) if row['model_name'] == 'LTX-Video-0.9.7-dev' else row['energy_generate_ram'],
    axis=1
)
df = combined_df_copy[['model_name', 'energy_generate_gpu', 'energy_generate_cpu', 'energy_generate_ram']]

# Grouper : moyenne et écart-type
grouped_mean = df.groupby("model_name").mean(numeric_only=True)
grouped_std = df.groupby("model_name").std(numeric_only=True)

# Conversion en Wh (multiplication par 1000)
grouped_mean *= 1000
grouped_std *= 1000

# Fusionner pour trier
grouped = grouped_mean.join(grouped_std, lsuffix='_mean', rsuffix='_std')

# Trier par consommation GPU moyenne décroissante
grouped = grouped.sort_values(by='energy_generate_gpu_mean', ascending=False).reset_index()

# Préparer données
models = grouped["model_name"]
gpu_mean = grouped["energy_generate_gpu_mean"]
cpu_mean = grouped["energy_generate_cpu_mean"]
ram_mean = grouped["energy_generate_ram_mean"]

gpu_std = grouped["energy_generate_gpu_std"]
cpu_std = grouped["energy_generate_cpu_std"]
ram_std = grouped["energy_generate_ram_std"]

# Bar plot
bar_width = 0.2
x = range(len(models))

plt.figure(figsize=(12, 10))

plt.bar([p - bar_width for p in x], gpu_mean, yerr=gpu_std, width=bar_width,
        label="GPU", color="#4E79A7", alpha=0.7, edgecolor="black", capsize=5)
plt.bar(x, cpu_mean, yerr=cpu_std, width=bar_width,
        label="CPU", color="#F28E2B", alpha=0.7, edgecolor="black", capsize=5)
plt.bar([p + bar_width for p in x], ram_mean, yerr=ram_std, width=bar_width,
        label="RAM", color="#59A14F", alpha=0.7, edgecolor="black", capsize=5)

# Labels et légende
plt.xlabel("Model")
plt.ylabel("Mean Energy Consumption (Wh)")
plt.title("Mean GPU, CPU, RAM Energy Consumption per Model (with Std Dev) in Wh")
plt.xticks(x, models, rotation=45, ha="right")
plt.legend()
plt.tight_layout()
plt.show()


TypeError: 'float' object is not subscriptable

In [64]:
# sort by model, prompt
save_df = combined_df[['model_name', 'duration_generate', 'energy_generate_gpu', 'energy_generate_cpu',
       'energy_generate_ram', 'prompt', 'cpu_count', 'cpu_model', 'gpu_count',
       'gpu_model', 'num_frames', 'runs', 'warmup', 'fps',  'output_path', 'name_csv', 'steps',
       'guidance_scale', 'negative_prompt', 'height', 'width', 'adapter_repo',
       'adapter_ckpt', 'base_model', 'duration_upsample',
       'duration_denoise', 'energy_upsample_gpu',
       'energy_denoise_gpu', 'energy_upsample_cpu', 'energy_denoise_cpu',
       'energy_upsample_ram', 'energy_denoise_ram', 'upsample_model_name',
       'downscaled_height', 'downscaled_width', 'generate_steps',
       'denoise_steps', 'denoise_strength', 'decode_timestep',
       'image_cond_noise_scale' ]]
save_df = save_df.sort_values(by=['model_name', 'prompt']).reset_index(drop=True)

In [65]:
save_df.to_csv(os.path.join(DATA_DIR, 'combined_results.csv'), index=False)

In [62]:
combined_df.columns

Index(['model_name', 'duration_generate', 'energy_generate_cpu',
       'energy_generate_ram', 'prompt', 'num_frames', 'runs', 'out_video',
       'out_csv', 'fps', 'warmup', 'output_path', 'name_csv', 'steps',
       'guidance_scale', 'negative_prompt', 'height', 'width', 'adapter_repo',
       'adapter_ckpt', 'base_model', 'out_gif', 'duration_upsample',
       'duration_denoise', 'energy_generate_gpu', 'energy_upsample_gpu',
       'energy_denoise_gpu', 'energy_upsample_cpu', 'energy_denoise_cpu',
       'energy_upsample_ram', 'energy_denoise_ram', 'upsample_model_name',
       'downscaled_height', 'downscaled_width', 'generate_steps',
       'denoise_steps', 'denoise_strength', 'decode_timestep',
       'image_cond_noise_scale', 'cpu_count', 'cpu_model', 'gpu_count',
       'gpu_model'],
      dtype='object')

In [55]:
combined_df[['model_name', 'energy_generate_gpu', 'energy_upsample_gpu',
       'energy_denoise_gpu', 'energy_upsample_cpu', 'energy_denoise_cpu',
       'energy_upsample_ram', 'energy_denoise_ram', 'upsample_model_name',
       'downscaled_height', 'downscaled_width', 'generate_steps',
       'denoise_steps', 'denoise_strength', 'decode_timestep',
       'image_cond_noise_scale']]

,model_name,energy_generate_gpu,energy_upsample_gpu,energy_denoise_gpu,energy_upsample_cpu,energy_denoise_cpu,energy_upsample_ram,energy_denoise_ram,upsample_model_name,downscaled_height,downscaled_width,generate_steps,denoise_steps,denoise_strength,decode_timestep,image_cond_noise_scale
0,AnimateDiff,0.000111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AnimateDiff,0.000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AnimateDiff,0.000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AnimateDiff,0.000110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AnimateDiff,0.000113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AnimateDiff,0.000113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AnimateDiff,0.000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AnimateDiff,0.000113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AnimateDiff,0.000111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AnimateDiff,0.000113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
